# Выявление особенностей словоупотребления в текстах Л. Н. Толстого методами дистрибутивной семантики

** TO DO **

- обучить сегментатор поточнее
- проверить его на "медленной" и "быстрой" версиях (по количеству предложений)
- проверить теги для дневников и писем
- почистить мета-информацию из дневников и писем
- "скормить" майстему дореволюционный (вроде ест)

In [10]:
import os
import re
from lxml import etree
from pymystem3 import Mystem
import string
import nltk
from langdetect import detect
import langid
from datetime import datetime
mystem = Mystem()

## Сегментатор

In [2]:
language_punkt_vars = nltk.tokenize.punkt.PunktLanguageVars
language_punkt_vars.sent_end_chars=('.','?','!','»','…')
with open('.\\data\\traindata.txt', 'r', encoding='utf-8') as f:
    traindata = f.read()
trainer = nltk.tokenize.punkt.PunktTrainer(traindata, language_punkt_vars)
trainer.INCLUDE_ALL_COLLOCS = True 
trainer.INCLUDE_ABBREV_COLLOCS = True
params = trainer.get_params()
sbd = nltk.tokenize.punkt.PunktSentenceTokenizer(params)

  Abbreviation: [0.6063] матф
  Abbreviation: [19.4117] см
  Abbreviation: [4.0590] г
  Abbreviation: [0.5493] стр
  Abbreviation: [5.4932] тыс
  Abbreviation: [4.0590] н
  Abbreviation: [1.0986] руб
  Abbreviation: [1.4932] ст
  Abbreviation: [4.0590] п
  Abbreviation: [5.9728] т.п
  Abbreviation: [4.0590] м
  Abbreviation: [1.4932] ул
0.00038153376573826786 0.007722007722007722 0.0 5242 259 2 2
0.00038153376573826786 0.007722007722007722 0.0 5242 259 2 2
0.003052270125906143 0.007722007722007722 0.0028095524784266506 5242 259 16 2
0.009538344143456695 0.03861003861003861 0.008027292795504716 5242 259 50 10
0.003052270125906143 0.007722007722007722 0.0028095524784266506 5242 259 16 2
0.0011446012972148034 0.007722007722007722 0.0008027292795504716 5242 259 6 2
0.0036245707745135446 0.007722007722007722 0.0034115994380895043 5242 259 19 2
0.0007630675314765357 0.007722007722007722 0.0004013646397752358 5242 259 4 2
0.0015261350629530714 0.023166023166023165 0.0004013646397752358 5242 2

## Данные

На основе 90-томного собрания сочинений

In [3]:
tei_ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

In [4]:
string.punctuation = string.punctuation.replace('-','')
string.punctuation

'!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

## Препроцессинг

In [5]:
def get_one_writing(filename):
    root = etree.parse(filename)#.getroot()
    for i in tags_for_deletion:
        for j in root.findall(i, tei_ns):
            parent = j.find("..")
            parent.remove(j)
    for i in tags_to_strip:
        etree.strip_tags(root, i)
    return root.find(".//tei:text", tei_ns).text.strip()

In [6]:
def preprocessing(text, f):
    #text = re.sub('[0-9—A-Za-zÉéÂâÊêÎîÔôÛûÀàÈèÙùËëÏïÜüŸÿÇç«»’§°¼½×áä˃<]+','', text)
    text = re.sub('[\n\s]+',' ', text)
    print (len(sbd.sentences_from_text(text, realign_boundaries=True)), end='\t')
    text = ' $$ '.join(sbd.sentences_from_text(text, realign_boundaries=True))
    text = text.replace('[','').replace(']','')
    text = ''.join(mystem.lemmatize(text.strip()))
    text = re.sub('[0-9—A-Za-zÉéÂâÊêÎîÔôÛûÀàÈèÙùËëÏïÜüŸÿÇç«»’§°¼½×áä˃<Α-Ωα-ωίϊΐόάέύϋΰήώΊΪΐΌΆΈΎΫΰΉΏ…“„]+','', text)
    text = text.split(' $$ ')
    for i in text:
        i2 = i.translate(i.maketrans('','',string.punctuation))
        i2 = re.sub('[\n\s]+',' ', i2)
        if langid.classify(i2)[0] not in ['fr','la','el'] and len(i2)>25:
            f.write(i2.strip()+'\n')
            pass

In [11]:
def universal_preprocessing(path, 
                            tags_for_deletion, 
                            tags_to_strip, 
                            filename):
    tags_for_deletion = tags_for_deletion
    tags_to_strip = tags_to_strip
    mystem = Mystem()
    start_gen = datetime.now()
    with open (filename, 'w', encoding='utf-8') as f:
        for root, dirs, files in os.walk (path):
                for fl in files :
                    start = datetime.now() 
                    preprocessing(get_one_writing(root+'\\'+fl), f)
                    print ('{}\t{}\t{}'.format(os.path.getsize(root+'\\'+fl)//1024, 
                                              datetime.now()-start,
                                              fl))
    print ('\n\ntotal time:\t\t {}', datetime.now()-start_gen)
    

### Произведения

** Плохие теги: **

- note
- add
- sic
- orig
- head
- del
- figure
- table


** Ненужные теги: **

- pb
- hi
- corr
- reg
- choice
- div
- p

In [38]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb']

In [39]:
path = '.\\data\\Final\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'writings.txt')

938	125	0:00:06.689582	Abrégé de l’Evangile 24.xml
21	8	0:00:01.711174	AMOUR DE LA PATRIE 1.xml
70	37	0:00:02.680727	Carthago delenda est 39.xml
87	32	0:00:02.234192	Carthago delenda est. Черновое 39.xml
322	95	0:00:03.650718	Letter to a Hindoo (Письмо к индусу) 37.xml
44	13	0:00:02.286501	NOTES SUR LE SECOND CHAPITRE DES «CARACTÈRES» DE LA BRUYÈRE 1.xml
28	13	0:00:02.411121	The hostelry (Гостиница) 37.xml
19	19	0:00:01.733315	[Carthago delenda est] («Жизнь, та форма жизни»...) 27.xml
9	5	0:00:01.761486	[LE PRÉSENT, LE PASSÉ ET LE FUTUR] 1.xml
9	9	0:00:01.625448	[«Давно позабыл я о счастьи ...»] 1.xml
49	29	0:00:02.060703	[«Дядинька Жданов и кавалер Чернов»] 3.xml
60	42	0:00:01.992051	[«Ежели бы человек не желал…»] 1.xml
18	12	0:00:01.808660	[«Ей, Марьяна, брось работу!..»] 1.xml
37	22	0:00:01.988302	[«Знание музыки - субъективное и объективное...»] 1.xml
729	330	0:00:07.512887	[«Начала» романа «Сто лет»] 17.xml
372	203	0:00:04.133552	[«Начала» романа времен Петра I. Азовские походы] 1

### Письма

opener
note
pb
closer

In [21]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table',
                      './/tei:opener', './/tei:closer']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb']

In [22]:
path = '.\\data\\LETTERS\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'letters.txt')

0	5	0:00:00.019999	letter0.xml
0	7	0:00:00.015999	letter1.xml
0	5	0:00:00.016011	letter10.xml
0	7	0:00:00.020001	letter100.xml


XMLSyntaxError: ID note1 already defined, line 95, column 931 (./data/LETTERS/Volume_59/letter101.xml, line 95)

In [19]:
path = '.\\data\\LETTERS\\'
for root, dirs, files in os.walk (path):
        for fl in files :
            print(root+'\\\\'+fl)

.\data\LETTERS\Volume_59\\letter0.xml
.\data\LETTERS\Volume_59\\letter1.xml
.\data\LETTERS\Volume_59\\letter10.xml
.\data\LETTERS\Volume_59\\letter100.xml
.\data\LETTERS\Volume_59\\letter101.xml
.\data\LETTERS\Volume_59\\letter102.xml
.\data\LETTERS\Volume_59\\letter103.xml
.\data\LETTERS\Volume_59\\letter104.xml
.\data\LETTERS\Volume_59\\letter105.xml
.\data\LETTERS\Volume_59\\letter106.xml
.\data\LETTERS\Volume_59\\letter107.xml
.\data\LETTERS\Volume_59\\letter108.xml
.\data\LETTERS\Volume_59\\letter11.xml
.\data\LETTERS\Volume_59\\letter12.xml
.\data\LETTERS\Volume_59\\letter13.xml
.\data\LETTERS\Volume_59\\letter14.xml
.\data\LETTERS\Volume_59\\letter15.xml
.\data\LETTERS\Volume_59\\letter16.xml
.\data\LETTERS\Volume_59\\letter17.xml
.\data\LETTERS\Volume_59\\letter18.xml
.\data\LETTERS\Volume_59\\letter19.xml
.\data\LETTERS\Volume_59\\letter2.xml
.\data\LETTERS\Volume_59\\letter20.xml
.\data\LETTERS\Volume_59\\letter21.xml
.\data\LETTERS\Volume_59\\letter22.xml
.\data\LETTERS\Volu

In [20]:
get_one_writing(".\data\LETTERS\Volume_59\\letter20.xml")
#get_one_writing("./data/Final/Abrégé de l’Evangile 24.xml")

XMLSyntaxError: ID note1 already defined, line 71, column 137 (./data/LETTERS/Volume_59/letter20.xml, line 71)

### Дневники

opener
note



div
p

In [23]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table', './/tei:opener']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb']

In [25]:
path = '.\\data\\diaries\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'diaries.txt')

0	981	0:00:00.403066	46.xml
0	698	0:00:00.093720	47.xml
0	268	0:00:00.046882	48.xml
0	425	0:00:00.063189	49.xml
0	778	0:00:00.080680	50.xml
0	441	0:00:00.062501	51.xml
0	612	0:00:00.084597	52.xml
0	897	0:00:00.146109	53.xml
0	752	0:00:00.149775	54.xml
0	1117	0:00:00.148735	55.xml
0	639	0:00:00.101028	56.xml
0	733	0:00:00.087278	57.xml
0	568	0:00:00.078128	58.xml


total time:		 {} 0:00:01.545688


In [28]:
path = '.\\data\\diaries\\'
for root, dirs, files in os.walk (path):
        for fl in files :
            print(root+'\\'+fl)

.\data\diaries\\46.xml
.\data\diaries\\47.xml
.\data\diaries\\48.xml
.\data\diaries\\49.xml
.\data\diaries\\50.xml
.\data\diaries\\51.xml
.\data\diaries\\52.xml
.\data\diaries\\53.xml
.\data\diaries\\54.xml
.\data\diaries\\55.xml
.\data\diaries\\56.xml
.\data\diaries\\57.xml
.\data\diaries\\58.xml


In [29]:
get_one_writing(".\data\diaries\\46.xml")

''

## Обработка текста

# Try

In [30]:
import gensim

C:\Users\Glaz\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [31]:
sentences = gensim.models.word2vec.LineSentence('writings.txt')

In [13]:
#sentences = [['я', 'иду'], ['я', 'иду']]

In [32]:
start = datetime.now() 
model = gensim.models.Word2Vec(sentences, min_count=1)
print(datetime.now()-start)

0:00:38.224809


In [33]:
print(model)

Word2Vec(vocab=47345, size=100, alpha=0.025)


In [34]:
print(model.wv.vocab.keys())

dict_keys(['в', '-м', 'год', 'святой', 'пасха', 'приходиться', 'апрель', 'весна', 'быть', 'ранний', 'и', 'дружный', 'неделя', 'проходить', 'река', 'бугор', 'на', 'страстной', 'еще', 'начинать', 'пахать', 'рожь', 'вскрываться', 'из', 'под', 'снег', 'зеленый', 'ровный', 'без', 'вымочка', 'зазеленеться', 'осенний', 'травка', 'становиться', 'пробивать', 'новый', 'скотина', 'уже', 'выпускать', 'выгон', 'мужик', 'поехать', 'свой', 'господский', 'радоваться', 'мягкий', 'рассыпчатый', 'разделка', 'земля', 'овсяный', 'посев', 'баба', 'девка', 'мужицкий', 'барский', 'двор', 'развешивать', 'по', 'плетень', 'платье', 'мыть', 'несбегать', 'снеговой', 'вода', 'порт', 'холст', 'кадушка', 'стол', 'лавка', 'готовиться', 'к', 'праздник', 'птица', 'не', 'разбираться', 'место', 'все', 'лететь', 'над', 'поле', 'лес', 'болото', 'кричать', 'ягненок', 'ржать', 'жеребенок', 'отыскивать', 'запрягать', 'соха', 'матка', 'чиж', 'жаворонок', 'щегленок', 'пеночка', 'со', 'свист', 'песня', 'перелетать', 'с', 'бабочка